In [1]:
import pandas as pd 
import urllib3
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
from datetime import datetime
from tqdm import tqdm
from datetime import date
import itertools

import warnings
warnings = 1

In [2]:
def convert_date(d):
    try:
        date = datetime.strptime(d, '%Y %b %d').strftime('%Y-%m-%d')
    except:
        date = datetime.strptime(d, '%Y %b').strftime('%Y-%m')
    return date

In [3]:
def clean_carriages(array):
    try:
        array = [line.replace('\r', '') for line in array]
    except:
        print('no return carriages')
    
    try:
        array = [line.replace('\n', '') for line in array]
    except:
        print('no newline carriages')
    
    return(array)

# might be able to use soup.stripped_strings:

In [4]:
def get_PI(masterList, grantNumber):
    a = []
    b = []
    temp = masterList[['Grant','PI']]

    for i in range(len(temp['Grant'])):
        if temp['Grant'][i] not in a:
            a.append(temp['Grant'][i])
            b.append(temp['PI'][i].split(',')[0])

    dic = dict(zip(a,b))

    return([*map(dic.get,grantNumber)])

    

In [5]:
def get_df_info(pubmed_url, ids, grant_names_1):
    url = pubmed_url + ids
    
    
    page = urlopen(url)
    html_doc = page.read()
    html = html_doc.decode("utf-8")

    # parse html content
    soup = BeautifulSoup( html_doc , 'html.parser')
    # =============================================== #
    # preprint or not:
    if (len(re.findall('preprint', str(soup))) < 1):
        print_type = ''
    else:
        print_type = 'preprint'
    
    # =============================================== #
    # Citation (since im lazy)
    try:
        citation = soup.find("div", {"class": "article-citation"}).text.replace('\r', '').replace('\n', '')
        citation = re.findall("(^.*?)(?:doi)", citation)[0].strip()
    except:
        citation = ''
    
    # =============================================== #
    # Finding by class name (title)
    title = soup.find(class_ = "heading-title")
    try:
        title = str(clean_carriages(title)[0]).strip()
    except:
        title = str(title)
        title = temp.replace('\n', '').replace('<h1 class="heading-title">','').replace('</h1>','')
        try:
            title = temp.replace('<i>', '').replace('</i>','').strip()

        except:
            print("no italics")
    
    # =============================================== #
    # Finding by class name (authors)
    author = soup.find_all(class_ = "authors-list-item")
    author_name = re.findall("data-ga-label=\"(.+?)\"", str(author))
    
    # =============================================== #
    # Finding date of publication (cit)
#     DOP = soup.find(class_ = "cit").string
#     try:
#         date = convert_date(DOP.split(';')[0])
#     except:
#         date = convert_date(DOP.split('.')[0])
    
    # =============================================== #
    # Finding the correct grant
    # parse html content
    soup = BeautifulSoup( html_doc , 'html.parser')
    grant = soup.find_all(class_ = "grant-item")
    grant = re.findall("data-ga-label=\"(.+?)\"", str(grant))

    str_grants = ', '.join(grant)
    # print(str_grants)
    
    # =============================================== #
    # clean out grants that are not in the right format:
    # bgrants = re.findall('(\w{3} \d{2}X\d{3})|(CA\d{6})', ' '.join(grant))
    bgrants = re.findall('(\w{3} CA\d{6})|(\w{3} \d{2}X\d{3})|(CA\d{6})', ' '.join(grant))
    out = list(filter(None,itertools.chain(*bgrants)))
    
    for i in out:
        base_grant = ''
        seroNet_grant = ''

        try:
            grant = i.split(' ')[1]
        except:
            grant = i

#         print(grant)

        try:
            if grant in grant_names_1:
                base_grant = grant
                seroNet_grant = grant
                break
        except:
            print('error')
    
    # Finding publishing journal 
    soup = BeautifulSoup(html_doc, 'lxml', from_encoding='utf-8')
    journal = re.findall('(?:meta content=\")(.*?)(?:name=\"citation_journal_title)',str(soup))[0][:-2]
    
    
    return title, author_name, journal, seroNet_grant, base_grant, print_type, citation
    # # Finding by class name (title)
    # title = soup.find(class_ = "heading-title")
    # title = str(clean_carriages(title)[0]).strip()

## Pull Papers ##

In [6]:
pmids_url = 'https://pubmed.ncbi.nlm.nih.gov/?term=CA260508+OR+CA260476+OR+CA260539+OR+CA' \
'60584+OR+CA260507+OR+CA260469+OR+CA260526+OR+CA260588+OR+CA260513+OR+CA260541+OR+CA260462'\
'+OR+CA261276+OR+CA261277+OR+CA260582+OR+CA260543+OR+CA260492+OR+CA260591+OR+CA260581+OR+' \
'CA260517+OR+CA260563+OR+CA260560+OR+21X089+OR+21X090+OR+21X091+OR+21X092%5BGrant+Number%5'\
'D&format=pubmed&sort=date&size=200'

master_list = pd.ExcelFile('/Users/liualg/National Institutes of Health/NCI-FNL SeroNet'\
                            ' Team - Curation channel/Master List Publications_2021-PMIDS and authors.xlsx')

grant_names = re.findall('(CA\d{6})', pmids_url)
grant_names += re.findall('(\d{2}X\d{3})', pmids_url)

In [7]:
PMID_input = '34314495 34605900 34910927 34931200 34937699 34960674 34965855'
PMID_list = PMID_input.split(' ')
search_url = 'https://pubmed.ncbi.nlm.nih.gov/'

In [8]:
PMID = []
title_list = []
author_list = []
journal_list = []
# date_list = []
grant_list = []
base_list = []
paperType_list = []
paper_citation = []
curator = []

for ids in tqdm(PMID_list):
#     print(ids)
    title, author_name, journal, seroNet_grant, base_grant, print_type, citation = get_df_info(search_url, ids, grant_names)
    
    PMID.append(ids)
    title_list.append(title)
    author_list.append(str(author_name))
    journal_list.append(journal)
#     date_list.append(date_str)
    grant_list.append(seroNet_grant)
    base_list.append(base_grant)
    paperType_list.append(print_type)
    paper_citation.append(citation)
    
    
grantCuratorDic = dict(zip(master_list.parse(sheet_name= 'curatorGrantMapping')['Grant'], 
                           master_list.parse(sheet_name= 'curatorGrantMapping')['primaryCurator']))

curator = [*map(grantCuratorDic.get,grant_list)]



100%|██████████| 7/7 [00:04<00:00,  1.46it/s]


In [9]:
empty = ['']*len(PMID)
updating_fields = {
    'Publication Owner':curator,
    'JIRA Epic #':empty,
    'Base_Project':base_list,
    'Status/ImmPort Study ID':empty,
    'Base_Project.1':grant_list, 
    'PMID':PMID,
    'PMID_N':['PMID_'+str(x) for x in PMID],
    'PMID_link':empty,
    'Publication_Title':title_list, 
    'Journal':journal_list,
    'Author(s)':author_list,
    'Pub Date':empty,
    'PI':empty,
    'print_type': paperType_list,
    'citation': paper_citation
}

In [13]:
temp = pd.DataFrame(updating_fields)

temp['Author(s)'] = temp['Author(s)'].str.replace('\'','')
temp['Author(s)'] = temp['Author(s)'].str.replace('[','')
temp['Author(s)'] = temp['Author(s)'].str.replace(']','')

temp['PI'] = get_PI(master_list.parse(sheet_name= 'curatorGrantMapping'), temp['Base_Project.1'])
today = date.today()
today = today.strftime("%y-%m-%d")
temp.to_csv(f'./new_pmids/data_pull_{today}.csv')
temp[['PMID','Publication_Title']].to_csv(f'./for_jenn/data_pull_{today}.csv')

temp.head()

<ipython-input-13-f166461bbbe6>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  temp['Author(s)'] = temp['Author(s)'].str.replace('[','')
<ipython-input-13-f166461bbbe6>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  temp['Author(s)'] = temp['Author(s)'].str.replace(']','')


,Publication Owner,JIRA Epic #,Base_Project,Status/ImmPort Study ID,Base_Project.1,PMID,PMID_N,PMID_link,Publication_Title,Journal,Author(s),Pub Date,PI,print_type,citation
0,Liu,,CA260517,,CA260517,34314495,PMID_34314495,,The Truth about SARS-CoV-2 Cycle Threshold Val...,Clinical chemistry,"Daniel D Rhoads, Benjamin A Pinsky",,BOYD,,Clin Chem. 2021 Dec 30;68(1):16-18.
1,Liu,,CA260517,,CA260517,34605900,PMID_34605900,,SARS-CoV-2 Nucleocapsid Plasma Antigen for Dia...,Clinical chemistry,"Hannah Wang, Catherine A Hogan, Michelle Vergh...",,BOYD,,Clin Chem. 2021 Dec 30;68(1):204-213.
2,Mani,,CA261277,,CA261277,34910927,PMID_34910927,,Population impact of SARS-CoV-2 variants with ...,Cell,"Mary Bushman, Rebecca Kahn, Bradford P Taylor,...",,LIPSITCH,,Cell. 2021 Dec 22;184(26):6229-6242.e18.
3,Liu,,CA260563,,CA260563,34931200,PMID_34931200,,Booster of mRNA-1273 Vaccine Reduces SARS-CoV-...,medRxiv : the preprint server for health sciences,"Nicole A Doria-Rose, Xiaoying Shen, Stephen D ...",,SANZ,preprint,medRxiv. 2021 Dec 15;2021.12.15.21267805.
4,Mani,,CA260582,,CA260582,34937699,PMID_34937699,,SARS-CoV-2 spreads through cell-to-cell transm...,Proceedings of the National Academy of Science...,"Cong Zeng, John P Evans, Tiffany King, Yi-Min ...",,OLTZ,,Proc Natl Acad Sci U S A. 2022 Jan 4;119(1):e2...


In [19]:
for i,o in enumerate(temp.PMID):
    if temp['print_type'][i] != 'preprint':
        print(f"CURATOR: {temp['Publication Owner'][i]}")
        print(temp['PMID_N'][i])
        print(f"*Title*: {temp['Publication_Title'][i]}")
        print(f"*Journal*: {temp['Journal'][i]}")
        print(f"*Pubmed*: https://pubmed.ncbi.nlm.nih.gov/{temp['PMID'][i]}")
        print(f"CURATOR: {temp['Publication Owner'][i]}")
        print(temp['PMID_N'][i])
        print(temp['PI'][i].capitalize())
        print(temp['Base_Project.1'][i])
        print(temp['PMID'][i])
        print("\n\n")

CURATOR: Liu
PMID_34314495
*Title*: The Truth about SARS-CoV-2 Cycle Threshold Values Is Rarely Pure and Never Simple
*Journal*: Clinical chemistry
*Pubmed*: https://pubmed.ncbi.nlm.nih.gov/34314495
CURATOR: Liu
PMID_34314495
Boyd
CA260517
34314495



CURATOR: Liu
PMID_34605900
*Title*: SARS-CoV-2 Nucleocapsid Plasma Antigen for Diagnosis and Monitoring of COVID-19
*Journal*: Clinical chemistry
*Pubmed*: https://pubmed.ncbi.nlm.nih.gov/34605900
CURATOR: Liu
PMID_34605900
Boyd
CA260517
34605900



CURATOR: Mani
PMID_34910927
*Title*: Population impact of SARS-CoV-2 variants with enhanced transmissibility and/or partial immune escape
*Journal*: Cell
*Pubmed*: https://pubmed.ncbi.nlm.nih.gov/34910927
CURATOR: Mani
PMID_34910927
Lipsitch
CA261277
34910927



CURATOR: Mani
PMID_34937699
*Title*: SARS-CoV-2 spreads through cell-to-cell transmission
*Journal*: Proceedings of the National Academy of Sciences of the United States of America
*Pubmed*: https://pubmed.ncbi.nlm.nih.gov/34937699
CUR

In [15]:
temp.sort_values(['Base_Project.1'])

,Publication Owner,JIRA Epic #,Base_Project,Status/ImmPort Study ID,Base_Project.1,PMID,PMID_N,PMID_link,Publication_Title,Journal,Author(s),Pub Date,PI,print_type,citation
0,Liu,,CA260517,,CA260517,34314495,PMID_34314495,,The Truth about SARS-CoV-2 Cycle Threshold Val...,Clinical chemistry,"Daniel D Rhoads, Benjamin A Pinsky",,BOYD,,Clin Chem. 2021 Dec 30;68(1):16-18.
1,Liu,,CA260517,,CA260517,34605900,PMID_34605900,,SARS-CoV-2 Nucleocapsid Plasma Antigen for Dia...,Clinical chemistry,"Hannah Wang, Catherine A Hogan, Michelle Vergh...",,BOYD,,Clin Chem. 2021 Dec 30;68(1):204-213.
5,Mani,,CA260541,,CA260541,34960674,PMID_34960674,,Limited Impact of Delta Variant's Mutations on...,Viruses,"Carlos A Sariol, Crisanta Serrano-Collazo, Edw...",,SARIOL,,Viruses. 2021 Nov 30;13(12):2405.
3,Liu,,CA260563,,CA260563,34931200,PMID_34931200,,Booster of mRNA-1273 Vaccine Reduces SARS-CoV-...,medRxiv : the preprint server for health sciences,"Nicole A Doria-Rose, Xiaoying Shen, Stephen D ...",,SANZ,preprint,medRxiv. 2021 Dec 15;2021.12.15.21267805.
4,Mani,,CA260582,,CA260582,34937699,PMID_34937699,,SARS-CoV-2 spreads through cell-to-cell transm...,Proceedings of the National Academy of Science...,"Cong Zeng, John P Evans, Tiffany King, Yi-Min ...",,OLTZ,,Proc Natl Acad Sci U S A. 2022 Jan 4;119(1):e2...
6,Mani,,CA260591,,CA260591,34965855,PMID_34965855,,Paradoxical sex-specific patterns of autoantib...,Journal of translational medicine,"Yunxian Liu, Joseph E Ebinger, Rowann Mostafa,...",,CHENG,preprint,J Transl Med. 2021 Dec 30;19(1):524.
2,Mani,,CA261277,,CA261277,34910927,PMID_34910927,,Population impact of SARS-CoV-2 variants with ...,Cell,"Mary Bushman, Rebecca Kahn, Bradford P Taylor,...",,LIPSITCH,,Cell. 2021 Dec 22;184(26):6229-6242.e18.
